In [1]:
import pandas as pd

In [ ]:
#!pip install --upgrade setuptools
#!pip install sklearn

In [2]:
repository = pd.read_csv("./autos.csv", encoding='ISO-8859-1')

In [3]:
repository = repository.drop('dateCrawled', axis=1)\
                       .drop('dateCreated', axis=1)\
                       .drop('nrOfPictures', axis=1)\
                       .drop('postalCode', axis=1)\
                       .drop('name', axis=1)\
                       .drop('seller', axis=1)\
                       .drop('offerType', axis=1)\
                       .drop('lastSeen', axis=1)

In [4]:
i1 = repository.loc[repository.price <= 10]
repository = repository[repository.price > 10]

In [5]:
i1 = repository.loc[repository.price >= 350000]
repository = repository[repository.price < 350000]

In [6]:
repository.loc[pd.isnull(repository['vehicleType'])]
repository['vehicleType'].value_counts() #limousine

limousine     93614
kleinwagen    78014
kombi         65921
bus           29699
cabrio        22509
coupe         18386
suv           14477
andere         3125
Name: vehicleType, dtype: int64

In [7]:
repository.loc[pd.isnull(repository['gearbox'])]
repository['gearbox'].value_counts() # manuell

manuell      266547
automatik     75508
Name: gearbox, dtype: int64

In [8]:
repository.loc[pd.isnull(repository['model'])]
repository['model'].value_counts() # golf

golf               28989
andere             25560
3er                19905
polo               12604
corsa              12149
                   ...  
serie_2                8
rangerover             6
serie_3                3
serie_1                1
discovery_sport        1
Name: model, Length: 251, dtype: int64

In [9]:
repository.loc[pd.isnull(repository['fuelType'])]
repository['fuelType'].value_counts() # benzin

benzin     217582
diesel     106002
lpg          5222
cng           557
hybrid        271
andere        165
elektro       101
Name: fuelType, dtype: int64

In [10]:
repository.loc[pd.isnull(repository['notRepairedDamage'])]
repository['notRepairedDamage'].value_counts() # nein

nein    259301
ja       34004
Name: notRepairedDamage, dtype: int64

In [11]:
values = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'model': 'golf',
    'fuelType': 'benzin',
    'notRepairedDamage': 'nein'
}

repository = repository.fillna(value=values)

In [12]:
previsors = repository.iloc[:, 1:13].values
real_price = repository.iloc[:, 0].values


In [13]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelEncoder_prev = LabelEncoder()
previsors[:,0] = labelEncoder_prev.fit_transform(previsors[:,0])
previsors[:,1] = labelEncoder_prev.fit_transform(previsors[:,1])
previsors[:,3] = labelEncoder_prev.fit_transform(previsors[:,3])
previsors[:,5] = labelEncoder_prev.fit_transform(previsors[:,5])
previsors[:,8] = labelEncoder_prev.fit_transform(previsors[:,8])
previsors[:,9] = labelEncoder_prev.fit_transform(previsors[:,9])
previsors[:,10] = labelEncoder_prev.fit_transform(previsors[:,10])

In [14]:
import numpy as np
encoder = OneHotEncoder(sparse=False)
cols_to_encode = [0, 1, 3, 5, 8, 9, 10]
columns_to_encode = previsors[:, cols_to_encode]
encoded_columns = encoder.fit_transform(columns_to_encode)
dataset_encoded = np.delete(previsors, cols_to_encode, axis=1)
dataset_encoded = np.concatenate((dataset_encoded, encoded_columns), axis=1)
previsorsEncoded = dataset_encoded.astype(float)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [15]:
previsorsEncoded.shape

(359291, 316)

In [16]:
from keras.layers import Dense
from keras.models import Sequential

In [17]:
regressor = Sequential()
regressor.add(Dense(units=158, activation='relu', input_dim=316))
regressor.add(Dense(units=158, activation='relu'))
regressor.add(Dense(units=1, activation='linear'))
regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
regressor.fit(previsorsEncoded, real_price, batch_size=300, epochs=100, workers=32, use_multiprocessing=True)

Epoch 1/100
1198/1198 [==============================] - 6s 4ms/step - loss: 3877.6445 - mean_absolute_error: 3877.6445
Epoch 2/100
1198/1198 [==============================] - 5s 4ms/step - loss: 3392.2302 - mean_absolute_error: 3392.2302
Epoch 3/100
1198/1198 [==============================] - 4s 3ms/step - loss: 3124.0164 - mean_absolute_error: 3124.0164
Epoch 4/100
1198/1198 [==============================] - 4s 3ms/step - loss: 2908.5283 - mean_absolute_error: 2908.5283
Epoch 5/100
1198/1198 [==============================] - 4s 3ms/step - loss: 2809.2256 - mean_absolute_error: 2809.2256
Epoch 6/100
 476/1198 [==========>...................] - ETA: 2s - loss: 2772.2151 - mean_absolute_error: 2772.2151

KeyboardInterrupt: 

In [ ]:
predictions = regressor.predict(previsorsEncoded)

11228/11228 [==============================] - 16s 1ms/step


In [ ]:
predictions

array([[  896.8269],
       [ 9207.505 ],
       [11572.95  ],
       ...,
       [ 6539.1523],
       [ 3537.3735],
       [25226.713 ]], dtype=float32)